In [70]:
#!pip install -U python-dotenv
#!pip install spotipy
#!pip install matplotlib

     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.6 MB 5.2 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/7.6 MB 11.4 MB/s eta 0:00:01
     -------- ------------------------------- 1.7/7.6 MB 15.2 MB/s eta 0:00:01
     ----------- ---------------------------- 2.1/7.6 MB 11.3 MB/s eta 0:00:01
     ----------------- ---------------------- 3.4/7.6 MB 14.4 MB/s eta 0:00:01
     ---------------------- ----------------- 4.3/7.6 MB 15.3 MB/s eta 0:00:01
     --------------------------- ------------ 5.3/7.6 MB 16.2 MB/s eta 0:00:01
     -------------------------------- ------- 6.3/7.6 MB 16.7 MB/s eta 0:00:01
     ---------------------------------------  7.5/7.6 MB 17.9 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.6 MB 18.1 MB/s eta 0:00:01
     ---------------------------------------- 7.6/7.6 MB 15.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/56.1 kB ? e


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\Jessi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os
import plotly.express as px
import spotipy
from spotipy import oauth2
from dotenv import load_dotenv
import os
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
path = os.getcwd()

# setting up authentication for spotipy
load_dotenv(dotenv_path=path + f"\env_vars.env")
os.environ['SPOTIPY_CLIENT_ID'] = os.environ.get('SPOTIPY_CLIENT_ID')
os.environ['SPOTIPY_CLIENT_SECRET'] = os.environ.get('SPOTIPY_CLIENT_SECRET')
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

### Part 1: Loading in the Data
#### We will load in my personal data blah blah, you can replace the string with your file path to analyze your music

In [4]:
num_histories = 4
listening = pd.DataFrame()
for i in range(num_histories):
    temp_df = pd.read_json(path + f"\MyData\StreamingHistory{i}.json")
    listening = pd.concat([listening, temp_df])
print(listening.shape)
listening.head(5)

(33284, 4)


,endTime,artistName,trackName,msPlayed
0,2020-12-26 00:02,Michael Schulte,The Love You Left Behind,194546
1,2020-12-26 00:05,Lewis Capaldi,Lost On You,197178
2,2020-12-26 00:08,Arlissa,Praying for Love,182535
3,2020-12-26 00:12,JP Saxe,A Little Bit Yours,225680
4,2020-12-26 00:16,My Brothers And I,Just Getting to Know You,252840


In [5]:
playlists = pd.read_json(path + f"\MyData\Playlist1.json")
playlists.head(5)

,playlists
0,"{'name': 'My Playlist #23', 'lastModifiedDate'..."
1,"{'name': 'My Playlist #22', 'lastModifiedDate'..."
2,"{'name': 'Stealing Spotify Daily Mixes', 'last..."
3,"{'name': 'Jess Chill Reccomended by Spotify', ..."
4,"{'name': 'Jess <3 Vibes?', 'lastModifiedDate':..."


### Part 2: EDA

In [6]:
playlists['playlists'][3]

{'name': 'Jess Chill Reccomended by Spotify',
 'lastModifiedDate': '2021-12-04',
 'items': [{'track': {'trackName': 'House',
    'artistName': 'Joshua Moss',
    'albumName': 'House',
    'trackUri': 'spotify:track:3BdtiKSZveePCDhfncBOFd'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Apple Pie',
    'artistName': 'Lizzy McAlpine',
    'albumName': 'Give Me A Minute',
    'trackUri': 'spotify:track:5FKNq0lLJnP3iZzBTD0PZj'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Free',
    'artistName': 'Elina',
    'albumName': 'Free',
    'trackUri': 'spotify:track:3hOWsTM9OpI6c72iQHP4EB'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Pancakes for Dinner',
    'artistName': 'Lizzy McAlpine',
    'albumName': 'Give Me A Minute',
    'trackUri': 'spotify:track:4HffeEF97c6UxNePgbuECW'},
   'episode': None,
   'localTrack': None}],
 'description': None,
 'numberOfFollowers': 0}

In [57]:
# information based on a single song
track_name, track_artist = 'Hey There Delilah', 'Plain White T\'s'
track_uri = sp.search(q='artist:' + track_artist + ' track:' + track_name, type='track')['tracks']['items'][0]['uri']
# track_analysis = sp.audio_analysis(track_uri) # too much info for me rn
track_features = sp.audio_features(track_uri)
track_features[0]

{'danceability': 0.656,
 'energy': 0.291,
 'key': 2,
 'loudness': -10.572,
 'mode': 1,
 'speechiness': 0.0293,
 'acousticness': 0.872,
 'instrumentalness': 0,
 'liveness': 0.114,
 'valence': 0.298,
 'tempo': 103.971,
 'type': 'audio_features',
 'id': '4RCWB3V8V0dignt99LZ8vH',
 'uri': 'spotify:track:4RCWB3V8V0dignt99LZ8vH',
 'track_href': 'https://api.spotify.com/v1/tracks/4RCWB3V8V0dignt99LZ8vH',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4RCWB3V8V0dignt99LZ8vH',
 'duration_ms': 232533,
 'time_signature': 4}

In [144]:
counts = listening.query('msPlayed >= 3000').groupby(by=['trackName', 'artistName']).size().reset_index()
songs_df = counts.rename(columns={0: 'count'}).sample(n=500, weights='count', random_state=42)#.query('count >= 5').sort_values('count', ascending=False) # either listened to >5 times or top x songs or random x songs
songs_df.head(5)#.to_csv('csv_exports/grouped.csv')

,trackName,artistName,count
2092,I Wish,Declan J Donovan,12
5201,friendship?,Jordy Searcy,10
3966,Sorry That I Do That,Anna Clendening,60
3232,One Day,Lovejoy,3
848,Complicated,Olivia O'Brien,6


In [145]:

lst_of_cols = list(track_features[0].keys())
# lst_of_cols.remove(['type', 'analysis_url'])
# lst_of_cols.remove(x for x in ['type', 'analysis_url'])
[lst_of_cols.remove(col) for col in ['type', 'uri', 'analysis_url', 'track_href']]
def find_info(song):
    # print(song)
    try:
        song_dict = sp.search(q='artist:' + song[1] + ' track:' + song[0], type='track')
        song_uri = song_dict['tracks']['items'][0]['uri']
        track_features = sp.audio_features(song_uri)[0]
        return pd.Series([track_features[i] for i in lst_of_cols])
    except:
        return pd.Series([None] * len(lst_of_cols))
    
    
songs_df[lst_of_cols] = songs_df[['trackName', 'artistName']].apply(find_info, axis=1)
print(songs_df.shape)
songs_df.head(5)

(500, 17)


,trackName,artistName,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
2092,I Wish,Declan J Donovan,12,0.625,0.241,7.0,-9.312,1.0,0.0308,0.84500,0.000000,0.1530,0.0960,111.902,7sXnAth7OEXjGhrW7Qsl3Y,198720.0,4.0
5201,friendship?,Jordy Searcy,10,0.571,0.224,4.0,-11.704,1.0,0.0646,0.69600,0.000000,0.1030,0.5060,108.526,0Io5aNyPCvK31w1hffcTbC,220133.0,4.0
3966,Sorry That I Do That,Anna Clendening,60,0.714,0.404,9.0,-8.759,1.0,0.0439,0.19600,0.000001,0.2830,0.2990,98.016,2WxCovJaXPloCHw0yJ0Z0z,149821.0,4.0
3232,One Day,Lovejoy,3,0.339,0.730,10.0,-8.568,0.0,0.0358,0.00737,0.000000,0.0472,0.8370,183.144,58U47mHjyLwUwJZ7voUt15,144375.0,4.0
848,Complicated,Olivia O'Brien,6,0.657,0.355,7.0,-12.047,1.0,0.0715,0.69400,0.000425,0.1230,0.0808,74.990,226PUvIuxGdceSijXXZRRz,189000.0,4.0


### PCA attempt/plotting the PCs/scree plot

In [97]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [146]:
cleaned_df = songs_df.set_index('trackName').drop(columns=['id', 'artistName']).dropna()
centered_df = (cleaned_df - np.mean(cleaned_df, axis=0)) / np.std(cleaned_df, axis=0)
print(centered_df.shape)
centered_df.head(5)

(476, 14)


,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
trackName,,,,,,,,,,,,,,
I Wish,-0.336546,0.561070,-0.801780,0.569724,0.142126,0.357718,-0.408420,0.857754,-0.206128,0.081054,-1.451574,-0.213323,-0.435917,0.338529
friendship?,-0.444422,0.143097,-0.884559,-0.282177,-0.478695,0.357718,0.236858,0.374627,-0.206128,-0.459821,0.905533,-0.329165,0.050218,0.338529
Sorry That I Do That,2.252479,1.249952,-0.008082,1.137657,0.285652,0.357718,-0.158327,-1.246604,-0.206119,1.487328,-0.284518,-0.689797,-1.546062,0.338529
One Day,-0.821988,-1.652640,1.579315,1.421624,0.335224,-2.795499,-0.312965,-1.858229,-0.206128,-1.063437,2.808466,2.231218,-1.669702,0.338529
Complicated,-0.660174,0.808758,-0.246678,0.569724,-0.567717,0.357718,0.368586,0.368142,-0.202324,-0.243471,-1.538959,-1.479893,-0.656589,0.338529


In [147]:
u, s, vt = np.linalg.svd(centered_df, full_matrices = False)
u.shape, s.shape, vt.shape

((476, 14), (14,), (14, 14))

In [148]:
np.round(s**2 / sum(s**2), 2)

array([0.2 , 0.12, 0.09, 0.08, 0.08, 0.07, 0.07, 0.06, 0.06, 0.05, 0.05,
       0.04, 0.03, 0.01])

In [149]:
# scree plot
px.line(s**2)

In [153]:
pcs = u * s
tst = centered_df.reset_index()[['trackName']]
tst['pc1'] = pcs[:,0]
tst['pc2'] = pcs[:,1]
tst['pc3'] = pcs[:,2]
px.scatter_3d(data_frame=tst, x='pc1', y='pc2', z='pc3', hover_data='trackName')
# pd.Index(cleaned_df).to_list()
# type(centered_df.reset_index()['id'])

In [282]:
# print(pcs)
all_pcs = pd.DataFrame(pcs)
all_pcs['trackName'] = centered_df.reset_index()[['trackName']]
all_pcs.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,trackName
0,0.882556,-0.256714,-1.209099,-0.076689,0.220817,-0.082461,0.361826,-0.443285,-0.063337,0.419308,-0.432925,1.095671,-0.813266,-0.166518,I Wish
1,0.461105,-0.902495,-0.132695,-0.471636,0.171530,0.709211,0.131362,-0.024040,0.437223,0.326097,0.337168,-0.611556,0.392517,-0.419933,friendship?
2,-0.900770,-0.896978,-0.888312,-0.309478,-1.382272,-1.472220,0.416125,0.417800,-2.343238,-0.792665,-0.621565,1.032822,0.437313,-0.587609,Sorry That I Do That
3,-2.701515,-0.178545,2.942639,0.811751,1.039950,-2.842454,0.604677,0.070731,1.078842,1.918193,-0.695688,-1.666992,1.428069,0.128843,One Day
4,0.478542,-0.829969,-1.750575,-0.020651,0.454270,0.311757,0.054733,-0.856634,0.214940,-0.539627,-0.901735,1.054224,0.119129,0.319593,Complicated


### Part 3: Analyzing/clustering the Data
#### PCA/plot/etc, figuring out what matters most in a song, what makes a song similar to another?

In [164]:
cluster_df = songs_df.set_index(['trackName', 'artistName', 'id']).dropna()
cluster_df = (cluster_df - np.mean(cluster_df, axis=0)) / np.std(cluster_df, axis=0)
cluster_df.head(5)

,,,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
trackName,artistName,id,,,,,,,,,,,,,,
I Wish,Declan J Donovan,7sXnAth7OEXjGhrW7Qsl3Y,-0.336546,0.561070,-0.801780,0.569724,0.142126,0.357718,-0.408420,0.857754,-0.206128,0.081054,-1.451574,-0.213323,-0.435917,0.338529
friendship?,Jordy Searcy,0Io5aNyPCvK31w1hffcTbC,-0.444422,0.143097,-0.884559,-0.282177,-0.478695,0.357718,0.236858,0.374627,-0.206128,-0.459821,0.905533,-0.329165,0.050218,0.338529
Sorry That I Do That,Anna Clendening,2WxCovJaXPloCHw0yJ0Z0z,2.252479,1.249952,-0.008082,1.137657,0.285652,0.357718,-0.158327,-1.246604,-0.206119,1.487328,-0.284518,-0.689797,-1.546062,0.338529
One Day,Lovejoy,58U47mHjyLwUwJZ7voUt15,-0.821988,-1.652640,1.579315,1.421624,0.335224,-2.795499,-0.312965,-1.858229,-0.206128,-1.063437,2.808466,2.231218,-1.669702,0.338529
Complicated,Olivia O'Brien,226PUvIuxGdceSijXXZRRz,-0.660174,0.808758,-0.246678,0.569724,-0.567717,0.357718,0.368586,0.368142,-0.202324,-0.243471,-1.538959,-1.479893,-0.656589,0.338529


In [178]:
# !pip install scikit-learn

In [194]:
# import scikit-learn as sklearn
from sklearn.cluster import MeanShift
from sklearn.cluster import KMeans

In [190]:
cluster = MeanShift().fit(cluster_df)
cluster.labels_

array([ 0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,
        0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  2,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  4,  0,  0,
        0,  0,  0,  2,  0,  0,  7,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  1,  0,  0,  0

In [193]:
cluster_w_labels = cluster_df.reset_index()
cluster_w_labels['color'] = cluster.labels_
# px.scatter(cluster_w_labels, x='count', y='energy', color='color', hover_data='trackName')
# cluster_w_labels.sort_values('color', ascending=False)

,trackName,artistName,id,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,color
252,I Have A Hole In My Tooth (And My Dentist is S...,dodie,025UUQA6NowbZynZU3Hv0u,-0.875926,1.304134,0.454503,-0.850111,-0.960661,0.357718,0.471678,-0.724567,7.553941,-0.775691,3.601834,1.430862,-4.039308,0.338529,14
130,Lovely,Lúc,4WjgUrjHyRebF9o0TvHVFc,-0.768050,-0.290357,1.871474,0.853691,0.972656,0.357718,7.919102,-0.341957,-0.206111,2.936872,-0.192534,0.379883,3.657331,0.338529,13
292,Pierre,Ryn Weaver,31c1V9xUONZYk78NE4Id8D,-0.714112,0.421746,2.051639,1.137657,1.447615,0.357718,-0.072418,-0.945055,-0.206102,6.192937,1.169989,-0.619592,-0.344493,0.338529,12
278,Forever & Always,Zeph,3aI4n3KHtrQ2MDeGmXhgWt,-0.552298,-0.429681,-1.473746,0.853691,-3.561257,-2.795499,-0.150691,1.324668,8.252078,-0.373281,-0.008564,1.261424,0.721608,0.338529,11
275,Stay,Kallista Rowan,13yLE339ZIunJQwKJctpMM,-0.444422,2.859924,-0.090860,0.853691,0.668215,0.357718,10.133665,-0.653233,-0.206128,-0.670762,2.503767,0.274679,-1.682438,0.338529,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Rich Boy,Sara Kays,6fAF2lOiZ86UNlY1Nth9tW,0.688276,-0.406461,0.264600,0.569724,0.299148,0.357718,0.465950,0.416779,-0.206128,-0.618838,0.980271,-0.382179,-0.923029,0.338529,0
162,When Someone Loves You,Tim Be Told,2Bl6jI3cxKKrSm5F2xNVHV,-0.552298,-0.669629,-1.603757,1.421624,-0.761594,0.357718,0.091765,1.162545,-0.206128,-0.200201,0.451359,0.322443,-1.361466,0.338529,0
161,You and I,Jess Ray,3rINQgcxDjNmTnUb3nDugO,-0.390484,0.785538,0.464242,-0.566144,0.198706,0.357718,-0.389329,-0.218743,-0.206128,-0.373281,2.279554,-0.313312,0.358976,0.338529,0
160,Broken,Jonah Kagen,44siIYlT1dMRQysOVjEvdF,3.493053,-0.398720,-0.280764,-1.134078,-0.539427,0.357718,0.108947,-0.257653,-0.206109,-0.427368,-0.416746,-1.186961,-0.511291,0.338529,0


In [241]:
cluster = KMeans(n_clusters=cluster_df.shape[0] // 10).fit_predict(cluster_df)
cluster_w_labels = cluster_df.reset_index()
cluster_w_labels['color'] = cluster
px.scatter(cluster_w_labels, x='count', y='energy', color='color', hover_data='trackName')

C:\Users\Jessi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [266]:
cluster = KMeans(n_clusters=tst.shape[0] // 10, n_init='auto').fit_predict(tst.drop(columns=['trackName']))
tst_plot = tst.reset_index()
tst_plot['color'] = cluster
px.scatter_3d(tst_plot, x='pc1', y='pc2', z='pc3', hover_data='trackName', color='color')

In [283]:
all_pc_cluster = KMeans(n_clusters=all_pcs.shape[0] // 10, n_init='auto').fit_predict(all_pcs.drop(columns=['trackName'])) #all_pcs
all_pc_plot = all_pcs
all_pc_plot['color'] = all_pc_cluster
px.scatter_3d(all_pc_plot, x=0, y=1, z=2, hover_data='trackName', color='color')

In [308]:
def get_playlist(song):
    # label = cluster_w_labels[cluster_w_labels['trackName'] == song].color.iloc[0] #.loc[0, 'color']
    # playlist = cluster_w_labels[cluster_w_labels['color'] == label]
    # return playlist['trackName']

    # # this is using the 3 PCs instead of raw elements, current least favorite bc it's less relevant
    # label = tst_plot[tst_plot['trackName'] == song].iloc[0, -1]#.loc['color', 0]
    # playlist = tst_plot[tst_plot['color'] == label]
    # return playlist['trackName']

    # this uses all the PCs instead of raw or 3 PCs
    label = all_pc_plot[all_pc_plot['trackName'] == song].iloc[0, -1]#.loc['color', 0]
    playlist = all_pc_plot[all_pc_plot['color'] == label]
    return playlist['trackName']
    

In [310]:
get_playlist('By Your Side')
# get_playlist('I Lived')
# get_playlist('Hey It\'s Delilah')
get_playlist('Hey There Delilah')

18                                       Hailey
22                  I Met Sarah in the Bathroom
64                             stuck in my head
70                       Somewhere in Ann Arbor
77                                         Home
106    The Way That I Love You (Single Version)
118                                      Ho Hey
122                        just like you - demo
165                                 Never the 1
174                         The Best - Acoustic
228                         Bloom - Bonus Track
261                                  Maybe I Do
291                               Please Notice
314                                    Hey Siri
324                                 Subway Song
329                       House With No Mirrors
342                           Hey There Delilah
404                                  Headlights
407                   Dancing In The Minefields
418             I Will Follow You into the Dark
445            Paper Hearts (feat. Baile

### Part 3: Loading in a playlist and comparing it to other songs in the playlist
#### Read in, PCA, plot with colors

### Part 4: Selecting additional relevant songs for playlist
#### Find most similar ones to playlist songs